In [ ]:
# for checking size match of input/output of autoencoder

In [1]:
# !pip install torchsummary

In [1]:
import numpy as np
import torch
from torchsummary import summary
from autoencoder import *

In [2]:
# model = Encoder(layers=[3, 16, 64, 128])
model = Encoder(layers=[3, 8], latent_dim=16)

In [3]:
x_in = np.random.random((10, 3, 128, 256)) # batch size, n channels, y, x
x_in.shape

(10, 3, 128, 256)

In [4]:
x_in = torch.from_numpy(x_in).type(torch.DoubleTensor)

In [5]:
output = model((x_in).float())

In [6]:
output.shape

torch.Size([10, 16, 32, 64])

In [7]:
# decoder = Decoder(layers=[3, 16, 64, 128])
decoder = Decoder(layers=[3, 8], latent_dim=16)

In [8]:
reconstruct = decoder(output)

In [9]:
reconstruct.shape

torch.Size([10, 3, 128, 256])

In [10]:
model = model.to('cuda')  # Move the model to the GPU
summary(model, (3, 128, 256), device="cuda")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 8, 64, 128]             224
              ReLU-2           [-1, 8, 64, 128]               0
            Conv2d-3           [-1, 16, 32, 64]           1,168
Total params: 1,392
Trainable params: 1,392
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.38
Forward/backward pass size (MB): 1.25
Params size (MB): 0.01
Estimated Total Size (MB): 1.63
----------------------------------------------------------------


In [11]:
decoder = decoder.to('cuda')  # Move the model to the GPU
summary(decoder, (16, 32, 64), device="cuda")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
          Upsample-1          [-1, 16, 64, 128]               0
            Conv2d-2           [-1, 8, 64, 128]           1,160
              ReLU-3           [-1, 8, 64, 128]               0
          Upsample-4          [-1, 8, 128, 256]               0
            Conv2d-5          [-1, 3, 128, 256]             219
              ReLU-6          [-1, 3, 128, 256]               0
Total params: 1,379
Trainable params: 1,379
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.12
Forward/backward pass size (MB): 5.50
Params size (MB): 0.01
Estimated Total Size (MB): 5.63
----------------------------------------------------------------


#### Full Implementation

In [26]:
# Example usage
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define layer sizes and initialize encoder/decoder
layer_sizes = [3, 8]  # Example layer sizes
encoder = Encoder(layers=layer_sizes, latent_dim=16).to(device)
decoder = Decoder(layers=layer_sizes, latent_dim=16).to(device)

# Initialize autoencoder
autoencoder = Autoencoder(encoder, decoder).to(device)

In [28]:
summary(autoencoder, (3, 128, 256), device="cuda")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 8, 64, 128]             224
              ReLU-2           [-1, 8, 64, 128]               0
            Conv2d-3          [-1, 16, 64, 128]           1,168
           Encoder-4          [-1, 16, 64, 128]               0
            Conv2d-5           [-1, 8, 64, 128]           1,160
          Upsample-6          [-1, 8, 128, 256]               0
            Conv2d-7          [-1, 3, 128, 256]             219
              ReLU-8          [-1, 3, 128, 256]               0
           Decoder-9          [-1, 3, 128, 256]               0
Total params: 2,771
Trainable params: 2,771
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.38
Forward/backward pass size (MB): 7.75
Params size (MB): 0.01
Estimated Total Size (MB): 8.14
-----------------------------------------------

In [27]:
pred = autoencoder((x_in).float())

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor